In [3]:
#!/usr/bin/env python3
# coding: utf-8
import sys
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model

optuna_csv = sys.argv[1]

with open('training.pkl', 'rb') as f:
     train = pickle.load(f)
    
with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)
        
with open('validation.pkl','rb') as f:
     val = pickle.load(f)
        
train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

train_photos =train_photos.astype('float32')
test_photos= test_photos.astype('float32')
val_photos =val_photos.astype('float32')
train_photos /= 255
val_photos /= 255
test_photos /= 255


In [4]:
from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna
import os
import tensorflow as tf


nb_classes = 2
epochs=3
batch_size =2

vgg16_model = VGG16(weights = 'imagenet', include_top = False)
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)
model = Model(input = vgg16_model.input, output = predictions)

for layer in vgg16_model.layers:
    layer.trainable = False
    
model.compile(optimizer = 'rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model_info = model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, 
                   verbose=1,validation_data=(val_photos,val_labels))

model.save('model_hpo.h5')

m= load_model("model_hpo.h5")
test_photos = test_photos.astype("float32")

results = m.evaluate(test_photos, test_labels, batch_size=1)
optkeras.optkeras.get_trial_default = lambda: optuna.trial.FrozenTrial(
            None, None, None, None, None, None, None, None, None, None, None)
study_name = "CatsAndDogs" + '_Simple'
ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')

def objective(trial):
    nb_classes = 2
    epochs=3
    batch_size =2

    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=trial.suggest_categorical('activation', ['relu', 'linear']))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)
    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = trial.suggest_categorical("optimizer", ["rmsprop", "Adam", "SGD"]),loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose ,validation_data=(val_photos,val_labels))
    return ok.trial_best_value

ok.optimize(objective, n_trials=3, timeout = 3*60)
import pandas as pd

pd.options.display.max_rows = 8 

pd.options.display.max_rows = 8

put_csv = pd.read_csv(ok.keras_log_file_path)
put_csv.to_csv(optuna_csv)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Train on 432 samples, validate on 48 samples
Epoch 1/3


426/432 [============================>.] - ETA: 1:28 - loss: 0.7249 - acc: 0.0000e+0 - ETA: 1:11 - loss: 2.3215 - acc: 0.2500    - ETA: 1:06 - loss: 1.5594 - acc: 0.500 - ETA: 1:03 - loss: 1.6531 - acc: 0.500 - ETA: 1:01 - loss: 1.4624 - acc: 0.500 - ETA: 59s - loss: 1.8200 - acc: 0.416 - ETA: 58s - loss: 1.6073 - acc: 0.50 - ETA: 57s - loss: 1.4133 - acc: 0.56 - ETA: 56s - loss: 1.2595 - acc: 0.61 - ETA: 55s - loss: 1.1372 - acc: 0.65 - ETA: 55s - loss: 1.3700 - acc: 0.59 - ETA: 54s - loss: 1.3276 - acc: 0.58 - ETA: 54s - loss: 1.2774 - acc: 0.57 - ETA: 53s - loss: 1.2554 - acc: 0.57 - ETA: 53s - loss: 1.2320 - acc: 0.53 - ETA: 53s - loss: 1.1616 - acc: 0.56 - ETA: 52s - loss: 1.1679 - acc: 0.55 - ETA: 52s - loss: 1.1439 - acc: 0.55 - ETA: 51s - loss: 1.0921 - acc: 0.57 - ETA: 51s - loss: 1.0408 - acc: 0.60 - ETA: 51s - loss: 0.9939 - acc: 0.61 - ETA: 51s - loss: 1.0218 - acc: 0.61 - ETA: 50s - loss: 1.0368 - acc: 0.58 - ETA: 50s - loss: 1.0302 - acc: 0.58 - ETA: 50s - loss: 1.0116 - 

426/432 [============================>.] - ETA: 1:09 - loss: 0.1218 - acc: 1.000 - ETA: 1:09 - loss: 0.1058 - acc: 1.000 - ETA: 1:11 - loss: 0.4810 - acc: 0.833 - ETA: 1:10 - loss: 0.5482 - acc: 0.750 - ETA: 1:10 - loss: 0.4778 - acc: 0.800 - ETA: 1:09 - loss: 0.4573 - acc: 0.833 - ETA: 1:08 - loss: 0.4009 - acc: 0.857 - ETA: 1:09 - loss: 0.3699 - acc: 0.875 - ETA: 1:08 - loss: 0.3407 - acc: 0.888 - ETA: 1:08 - loss: 0.3790 - acc: 0.850 - ETA: 1:07 - loss: 0.4403 - acc: 0.772 - ETA: 1:06 - loss: 0.4060 - acc: 0.791 - ETA: 1:06 - loss: 0.4206 - acc: 0.769 - ETA: 1:05 - loss: 0.3982 - acc: 0.785 - ETA: 1:04 - loss: 0.4171 - acc: 0.766 - ETA: 1:04 - loss: 0.4154 - acc: 0.781 - ETA: 1:03 - loss: 0.4115 - acc: 0.794 - ETA: 1:03 - loss: 0.4064 - acc: 0.805 - ETA: 1:03 - loss: 0.4158 - acc: 0.789 - ETA: 1:03 - loss: 0.4560 - acc: 0.750 - ETA: 1:03 - loss: 0.4518 - acc: 0.738 - ETA: 1:03 - loss: 0.4334 - acc: 0.750 - ETA: 1:03 - loss: 0.4257 - acc: 0.760 - ETA: 1:03 - loss: 0.4111 - acc: 0.770

432/432 [==============================] - ETA: 0s - loss: 0.4250 - acc: 0.787 - ETA: 0s - loss: 0.4235 - acc: 0.788 - 79s 183ms/step - loss: 0.4266 - acc: 0.7870 - val_loss: 0.8112 - val_acc: 0.6042
Epoch 3/3


...

[2020-09-08 13:44:37.389576] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:69: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


[2020-09-08 13:44:38.171433]  (None) 


[I 2020-09-08 13:48:47,132] Trial 0 finished with value: 0.6666666666666666 and parameters: {'activation': 'linear', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.6666666666666666.


[2020-09-08 13:48:47.217852] Trial#: 0, value: 6.666667e-01| Best trial#: 0, value: 6.666667e-01, params: {'activation': 'linear', 'optimizer': 'SGD'}


In [5]:
ok.optimize(objective, n_trials=4, timeout = 3*60)
import pandas as pd

pd.options.display.max_rows = 8 

put_csv = pd.read_csv(ok.keras_log_file_path)
put_csv.to_csv(optuna_csv)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:69: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


[2020-09-08 13:48:48.189721] Trial#: 0, value: 6.666667e-01| Best trial#: 0, value: 6.666667e-01, params: {'activation': 'linear', 'optimizer': 'SGD'}


[I 2020-09-08 13:53:39,267] Trial 1 finished with value: 0.8958333333333334 and parameters: {'activation': 'linear', 'optimizer': 'Adam'}. Best is trial 1 with value: 0.8958333333333334.


[2020-09-08 13:53:39.355121] Trial#: 1, value: 8.958333e-01| Best trial#: 1, value: 8.958333e-01, params: {'activation': 'linear', 'optimizer': 'Adam'}


In [14]:
import argparse
import joblib
MODEL = "basicnet"
EPOCHS = 5
CHECKPOINTS_FILES_LIST = []
STUDY = None
N_TRIALS = 5
BATCH_SIZE = 4

parser = argparse.ArgumentParser(description='Binary Classification of Images')
parser.add_argument('epochs',  metavar='num_epochs', type=int, nargs=1, help = "number of training epochs")
parser.add_argument('trials',  metavar='num_trials', type=int, nargs=1, help = "number of HPO trials")

def hpo_monitor(study, trial):
    joblib.dump(study,"hpo_study_checkpoint_" + MODEL + ".pkl")
    
def hpo_monitor_sigterm(final_prefix = ""):
    joblib.dump(STUDY,final_prefix + "hpo_study_checkpoint_" + MODEL + ".pkl")
    
def objective(trial):
    nb_classes = 2
    epochs=3
    batch_size =2
    optimizer_options = ["RMSprop", "Adam", "SGD"]
    losses_dict= {'train': {}, 'test': {}, 'accuracy': {}}

    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=trial.suggest_categorical('activation', ['relu', 'linear']))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)
    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = trial.suggest_categorical("optimizer", ["rmsprop", "Adam", "SGD"]),loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose ,validation_data=(val_photos,val_labels))
    return ok.trial_best_value

def check_status_last_trial():
    global STUDY
    last_trial_index = len(STUDY.trials) -1
    last_trial_status = str(STUDY.trials[last_trial_index].state)
    
def create_study(hpo_checkpoint_file):
    global STUDY
    load_checkpoint_flag = True
    
    if len(CHECKPOINTS_FILES_LIST) == 0:
        load_checkpoint_flag = False

    if load_checkpoint_flag:
        #print("Loading an existing study...")
        STUDY = joblib.load("hpo_study_checkpoint_" + MODEL + ".pkl")
        #check_status_last_trial()
        todo_trials = N_TRIALS - len(STUDY.trials_dataframe())
        if todo_trials > 0 :
            #print("There are {} trials to do out of {}".format(todo_trials, N_TRIALS))
            STUDY.optimize(objective, n_trials=todo_trials, timeout=600, callbacks=[hpo_monitor])
        else:
            pass
            #print("This study is finished. Nothing to do.")
    else:
        STUDY = optuna.create_study(direction = 'maximize', study_name = MODEL)
        STUDY.optimize(objective, n_trials=N_TRIALS, timeout=600, callbacks=[hpo_monitor])

def main():

    global MODEL
    global EPOCHS
    global CHECKPOINTS_FILES_LIST
    global N_TRIALS

    try:
        signal.signal(signal.SIGTERM, sigterm_handler)
        
        args     = parser.parse_args()
        EPOCHS   = args.epochs[0]
        N_TRIALS = args.trials[0]

        hpo_checkpoint_file = "hpo_study_checkpoint_" + MODEL + ".pkl"
        create_study(hpo_checkpoint_file)

    except Exception as e:
    	pass
    finally:
        hpo_monitor_sigterm("final_")

  
    return 0

if __name__ == '__main__':
    main()

        